První pokusy


In [3]:
pip install psycopg2-binary


     |████████████████████████████████| 3.0 MB 4.7 MB/s eta 0:00:01     |████████                        | 737 kB 4.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install -U pip

Note: you may need to restart the kernel to use updated packages.


In [1]:
#NEJAKY ERROR ???? pred tim jelo a melo by jet !
print('kontrola v listu')
pip list

SyntaxError: invalid syntax (<ipython-input-1-73d796a36bfd>, line 3)

<l1>Kontrola knihovny</l1>

<l2><b>Prvotni pripojeni k databazi (manualne) </b></l2>

Pozdeji vytvorit config file s pripojenim - lepsi pri zmene databaze atd... <a href = https://www.postgresqltutorial.com/postgresql-python/connect/>(priklad, v 1/3 stranky) </a>

In [7]:
import psycopg2

print('Manual connecting to the PostgreSQL database...')
conn = psycopg2.connect(
    host="postgres",
    database="dvdrental",
    user="postgres",
    password="example")

# create a cursor
cur = conn.cursor()
 
# execute a statement
print('PostgreSQL database version:')
cur.execute('SELECT version()')

# display the PostgreSQL database server version
db_version = cur.fetchone() #fetchone()->metoda
print(db_version)

# close the communication with the PostgreSQL
cur.close()


Manual connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 13.2 (Debian 13.2-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',)


<b> Ukonceni spojeni s databazi </b>

In [8]:
conn.close()
print('Database connection closed.')

Database connection closed.


<b> Zkouska vytvoreni tabulky PANDA z dat SQL databaze na pgAdmin postgressu</b>

In [6]:
import psycopg2
import pandas as pd

print('Manual connecting to the PostgreSQL database...')
conn = psycopg2.connect(
    host="postgres",
    database="dvdrental",
    user="postgres",
    password="example")

# create a cursor
cur = conn.cursor()

# A function that takes in a PostgreSQL query and outputs a pandas database
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table
 
# Utilize the create_pandas_table function to create a Pandas data frame
# Store the data as a variable
actor_info = create_pandas_table("SELECT actor_id, first_name, last_name FROM actor ORDER BY actor_id")
actor_info

film_info = create_pandas_table("SELECT film_id, title, release_year FROM film ORDER BY title") 
#film_info

# close the communication with the PostgreSQL
cur.close()
conn.close()

print (actor_info)
print('------FILMY------TEST------\n')
print(film_info[:1000])

Manual connecting to the PostgreSQL database...
     actor_id first_name     last_name
0           1   Penelope       Guiness
1           2       Nick      Wahlberg
2           3         Ed         Chase
3           4   Jennifer         Davis
4           5     Johnny  Lollobrigida
..        ...        ...           ...
195       196       Bela        Walken
196       197      Reese          West
197       198       Mary        Keitel
198       199      Julia       Fawcett
199       200      Thora        Temple

[200 rows x 3 columns]
------FILMY------TEST------

     film_id              title  release_year
0          1   Academy Dinosaur          2006
1          2     Ace Goldfinger          2006
2          3   Adaptation Holes          2006
3          4   Affair Prejudice          2006
4          5        African Egg          2006
..       ...                ...           ...
995      996     Young Language          2006
996      997         Youth Kick          2006
997      998     

<b> FAST API ????</b>


Pro splnění aby nebyl přistup jen z root adresáře:
    

In [28]:
from fastapi import FastAPI, Request

app = FastAPI()
@app.get("/")
async def root():
    return {"message" : "Hello world"}

@app.get("/items/{item_id}")
async def read_item(item_id: int):
    return {"item_id": item_id}


apiapp = FastAPI()

@apiapp.get("/items/{item_id}")
async def apif_read_item(item_id: int):
    return {"item_id": item_id}

@apiapp.get("/")
async def apif_root():
    return {"message": "Hello world"}

app.mount('/apif', apiapp)

<b>FAST api TESTOVÁNÍ jak to funguje </b>

In [4]:
!pip install uvicorn
!pip install fastapi

In [24]:
#spusteni FASTApi

import uvicorn
from fastapi import FastAPI

app = FastAPI()#root_path='/api') #vytovreni app

def run(): #funkce spusteni
    uvicorn.run(app, port=9992, host='0.0.0.0', root_path='')  #HOST je NUTNY PARAMETR !!! 

In [25]:
# Code in this cell is just for (re)starting the API on a Process, and other compatibility stuff with Jupyter cells.
# Just ignore it!

from multiprocessing import Process
from wait4it import wait_for

_api_process = None

def start_api(runNew=True):
    """Stop the API if running; Start the API; Wait until API (port) is available (reachable)"""
    global _api_process
    if _api_process:
        _api_process.terminate()
        _api_process.join()
    
    if runNew:
        _api_process = Process(target=run, daemon=True)
        _api_process.start()
        wait_for(port=9992)

def delete_route(method: str, path: str):
    """Delete the given route from the API. This must be called on cells that re-define a route"""
    [app.routes.remove(route) for route in app.routes if method in route.methods and route.path == path]
    

In [26]:
def delete_all_routes():
    rr = [*app.routes]
    for item in rr:
        app.routes.remove(item)

In [30]:
# TEST deklarace ENDpointů

@app.get("/api")
def get_root(): 
    return {"Hello": "World"}

@app.get("/items/{item_id}")
async def read_item(item_id: int):
    return {"item_id": item_id}

apiapp = FastAPI()

@apiapp.get("/items/{item_id}")
async def apif_read_item(item_id: int):
    return {"item_id": item_id}

@apiapp.get("/")
async def apif_root():
    return {"message": "Hello world"}

app.mount('/apif', apiapp)

start_api() # samotne spusteni

INFO:     Started server process [160]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9992 (Press CTRL+C to quit)


INFO:     172.18.0.1:37916 - "GET /docs HTTP/1.1" 200 OK
INFO:     172.18.0.1:37916 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     172.18.0.1:37914 - "GET /items/5 HTTP/1.1" 200 OK
INFO:     172.18.0.1:37936 - "GET /docs HTTP/1.1" 200 OK
INFO:     172.18.0.1:37936 - "GET / HTTP/1.1" 200 OK
INFO:     172.18.0.1:37942 - "GET /api HTTP/1.1" 200 OK
INFO:     172.18.0.1:37952 - "GET /items/4 HTTP/1.1" 200 OK
INFO:     172.18.0.1:37960 - "GET /items/sas HTTP/1.1" 422 Unprocessable Entity
INFO:     172.18.0.1:37974 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [160]


In [31]:
start_api(False) #zastaveni serveru